In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import Packages

In [ ]:
!pip install seaborn
!pip install bubbly
!pip install plotly
!pip install IPython
!pip install eli5
!pip install shap
!pip install pdpbox

In [ ]:
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
from bubbly.bubbly import bubbleplot
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
import eli5
from eli5.sklearn import PermutationImportance
import shap
from pdpbox import pdp, info_plots
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import tree
import numpy as np
from sklearn.tree import export_graphviz
import graphviz

# Data description(from UCI)
- data size : (303, 14), Number of instances : 303 , feature : 14
- missing values : None 
- Data source url : archive.ics.uci.edu/ml/datasets/heart + disease 
- This database contains 76 attributes, but all published experiments refer to using a subset
  of 14 of them. In particular, the Cleveland database is the only one that has been used by
  ML researchers to this date. The "goal" field refers to the presence of heart disease
  in the patient. It is integer valued from 0 (no presence) to 4. Experiments with
  the Cleveland database have concentrated on simply attempting to distinguish presence
  (values 1,2,3,4) from absence (value 0). The names and social security numbers of the patients
  were recently removed from the database, replaced with dummy values.

# Features description
### Stat description
       1. age : interval scale, values = [29, 77] , dtypes = int
       2. sex : nominal scale, values = 0,1
       3. cp : nominal scale, values = 0,1,2,3
       4. trestbps : interval scale, values = [94, 200] , dtypes = int
       5. chol : interval scale, values = [126, 564] , dtypes = int
       6. fbs : nominal scale, values = 0,1
       7. restecg : nominal scale, values = 0,1,2
       8. thalach : interval scale, value = [71, 202], dtypes = int
       9. exang : nominal scale, value = 0,1
      10. oldpeak : interval scale, value = [0, 6.2] dtypes = float
      11. slope : nominal scale, value = 0,1,2
      12. ca : nominal scale, value = 0,1,2,3
      13. thal : nominal scale, value = 0,1,2,3
      14. target : nominal scale, value = 0,1


### feature definition
 - age : 29~77                                                                  
 - sex : 0 = female, 1 = male                                                   
 - cp : chest pain type(category=0,1,2,3)                                       
  -- Value 0: typical angina                                                 
  -- Value 1: atypical angina                                                
  -- Value 2: non-anginal pain                                               
  -- Value 3: asymptomatic                                                   
 - trestbps : resting blood pressure(in mm Hg on admission to the hospital)     
 - chol : serum cholestoral in mg/dl                                             
 - fbs : fasting blood sugar >120 mg/dl (1=True, 0=False)                       
 - restecg : resting electrocardiographic result(category=0,1,2)  
  -- Value 0: normal                                                         
  -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)   
  -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria                       
 - thalach : maximum heart rate achieved                                        
 - exang : exercise induced angina (1=yes, 0=no)                                
 - oldpeak : ST depression induced by exercise relative to rest                 
 - slope : the slope of the peak exercise ST segment(category:0,1,2)  
  -- Value 1: upsloping                                                      
  -- Value 2: flat                                                           
  -- Value 3: downsloping                                                    
 - ca : number of major vessels(0-3) colored by flourosopy                      
 - thal : A blood disorder called thalassemia                                   
3 =normal, 6=fixed defect, 7= reversable defect  
 - target : yes=1 or no=0 diagnosis of heart disease (angiographic disease status)   
  -- Value 0: < 50% diameter narrowing                                         
  -- Value 1: > 50% diameter narrowing                                        

# Data shape

In [ ]:
dt = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')
dt.head()

# Data Background Knowledge
## [electrocardiographic normal]
![image](https://w7.pngwing.com/pngs/160/614/png-transparent-qt-interval-long-qt-syndrome-pr-interval-t-wave-qrs-complex-electrocardiogram-angle-text-heart-thumbnail.png)

## feature(slope and restecg)  
### slope : S point ~ T point's inclination
### restecg : This is the result of making a judgment by looking at the graph.

# Data preprocessing

In [ ]:
# data shape
dt.shape

In [ ]:
# Check data type
dt.info()

In [ ]:
# Change columns's name generally
dt.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved',
       'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

In [ ]:
# Divided features by scale
category_ft_dt = dt[['sex','chest_pain_type','fasting_blood_sugar','rest_ecg','exercise_induced_angina','st_slope','num_major_vessels','thalassemia','target']]
numeric_ft_dt = dt[['age','resting_blood_pressure','cholesterol','max_heart_rate_achieved','st_depression','target']]
target = dt[['target']]

In [ ]:
print('category features','\n',category_ft_dt.head())
print('numeric features','\n',numeric_ft_dt.head())

# Data EDA

In [ ]:
# categories value change string
eda_heart1=dt.copy()
eda_heart1['sex'][eda_heart1['sex']==0] ='female'
eda_heart1['sex'][eda_heart1['sex']==1] ='male'
eda_heart1['chest_pain_type'][eda_heart1['chest_pain_type']==0] = 'typical angina'
eda_heart1['chest_pain_type'][eda_heart1['chest_pain_type']==1] = 'atypical angina'
eda_heart1['chest_pain_type'][eda_heart1['chest_pain_type']==2] = 'non-anginal pain'
eda_heart1['chest_pain_type'][eda_heart1['chest_pain_type']==3] = 'asymptomatic'
eda_heart1['fasting_blood_sugar'][eda_heart1['fasting_blood_sugar']==0] = 'lower than 120mg/ml'
eda_heart1['fasting_blood_sugar'][eda_heart1['fasting_blood_sugar']==1] = 'greater than 120mg/ml'
eda_heart1['rest_ecg'][eda_heart1['rest_ecg']==0] = 'normal'
eda_heart1['rest_ecg'][eda_heart1['rest_ecg']==1] = 'ST-T wave abnormality'
eda_heart1['rest_ecg'][eda_heart1['rest_ecg']==2] = 'left ventricular hypertrophy'
eda_heart1['exercise_induced_angina'][eda_heart1['exercise_induced_angina']==0] = 'no'
eda_heart1['exercise_induced_angina'][eda_heart1['exercise_induced_angina']==1] = 'yes'
eda_heart1['st_slope'][eda_heart1['st_slope']==0] = 'upsloping'
eda_heart1['st_slope'][eda_heart1['st_slope']==1] = 'flat'
eda_heart1['st_slope'][eda_heart1['st_slope']==2] = 'downsloping'
eda_heart1['thalassemia'][eda_heart1['thalassemia']==0] = 'Unknown'
eda_heart1['thalassemia'][eda_heart1['thalassemia']==1] = 'normal'
eda_heart1['thalassemia'][eda_heart1['thalassemia']==2] = 'fixed defect'
eda_heart1['thalassemia'][eda_heart1['thalassemia']==3] = 'reversable defect'
eda_heart1['target'][eda_heart1['target']==0] = '< 50% diameter narrowing'
eda_heart1['target'][eda_heart1['target']==1] = '> 50% diameter narrowing'

In [ ]:
eda_heart1.head()

## Graphing nominal data

In [ ]:
plt.rcParams['axes.axisbelow'] = True
plt.figure(figsize=(30,30))
for i in range(len(category_ft_dt.columns)):
    plt.subplot(3,3,i+1)
    sns.countplot(eda_heart1[category_ft_dt.columns[i]])
    plt.xlabel(category_ft_dt.columns[i],size=20)
    plt.ylabel('count',size=20, rotation=0,labelpad=30)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.grid(axis='y')

In [ ]:
print('category data ratio','\n')
for i in category_ft_dt.columns:
    print(i, '\n')
    for j in eda_heart1[i].value_counts().index:
        print(j,' : ',round(eda_heart1[i].value_counts()[j]/eda_heart1.shape[0]*100,2),'%')
    print()

# category feature's target value ratio

In [ ]:
xlabeling=['sex','chest_pain_type','fasting_blood_sugar','rest_ecg','exercise_induced_angina','st_slope','num_major_vessels','thalassemia']
for i in range(len(xlabeling)):
    f,ax = plt.subplots(figsize=(10,8))
    sns.countplot(x=xlabeling[i],hue="target",data=eda_heart1)
    bars = ax.patches
    half = int(len(ax.patches)/2)
    plt.xlabel(xlabeling[i],size=20)
    plt.ylabel('count',size=20, rotation=0,labelpad=30)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(loc=1,prop={'size':20})
    plt.grid(axis='y')

    for first,second in zip(bars[:half],bars[half:]):
        height1 =  first.get_height()
        height2 = second.get_height()
        total_height= height1+height2
        ax.text(first.get_x()+first.get_width()/2, height1+1,'{0:.0%}'.format(height1/total_height), ha ='center')
        ax.text(second.get_x()+second.get_width()/2, height2+1,'{0:.0%}'.format(height2/total_height), ha ='center')

From this data, it was confirmed that there are two thalassemia's unknown.  
Unknown target ratio = 1:1.  
so these outliers subtract from the data.


## Graphing interval data

### Box plot

In [ ]:
labeling = ['age','resting_blood_pressure','cholesterol','max_heart_rate_achieved','st_depression']
graph_title=['age','resting_blood_pressure','cholesterol','max_heart_rate_achieved','st_depression']
y_labeling = ['age','mm/hg','mg/dl','heart_rate','inclination']
plt.figure(figsize=(40,20))
for i in  range(len(labeling)):
    plt.subplot(2,3,i+1)
    sns.boxplot('target',labeling[i],data=numeric_ft_dt)
    plt.title(graph_title[i],size=30)
    plt.xlabel('target',size=20)
    plt.ylabel(y_labeling[i],size=20, rotation=0,labelpad=53)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

In [ ]:
numeric_ft_dt.drop(['target'],axis=1).describe()

# Interval estimation

In [ ]:
eda_heart2=numeric_ft_dt.copy()
eda_heart2['interval_age']=pd.cut(eda_heart2.age,bins=[28,38,48,58,68,78])
eda_heart2['interval_resting_blood_pressure']=pd.cut(eda_heart2.resting_blood_pressure,bins=range(min(eda_heart2.resting_blood_pressure),max(eda_heart2.resting_blood_pressure)+20,20))
eda_heart2['interval_cholesterol']=pd.cut(eda_heart2.cholesterol,bins=range(min(eda_heart2.cholesterol),max(eda_heart2.cholesterol)+100,100))
eda_heart2['interval_max_heart_rate_achieved']=pd.cut(eda_heart2.max_heart_rate_achieved,bins=range(min(eda_heart2.max_heart_rate_achieved),max(eda_heart2.max_heart_rate_achieved)+20,20))
eda_heart2['interval_st_depression']=pd.cut(eda_heart2.st_depression,bins=[-0.1,1,2,3,4,5,6,7])

In [ ]:
eda_heart2['target_name']=eda_heart1['target']
labeling = ['interval_age','interval_resting_blood_pressure','interval_cholesterol','interval_max_heart_rate_achieved','interval_st_depression']
for i in range(len(labeling)):
    f,ax = plt.subplots(figsize=(10,8))
    sns.countplot(x=labeling[i],hue="target_name",data=eda_heart2)
    bars = ax.patches
    half = int(len(ax.patches)/2)
    plt.title(graph_title[i],size=30)
    plt.xlabel(labeling[i],size=20)
    plt.ylabel('count',size=20, rotation=0,labelpad=30)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(loc=1,prop={'size':20})
    plt.grid(axis='y')

    for first,second in zip(bars[:half],bars[half:]):
        height1 =  first.get_height()
        height2 = second.get_height()
        total_height= height1+height2
        ax.text(first.get_x()+first.get_width()/2, height1+1,'{0:.0%}'.format(height1/total_height), ha ='center')
        ax.text(second.get_x()+second.get_width()/2, height2+1,'{0:.0%}'.format(height2/total_height), ha ='center')

In [ ]:
# nan% check
for i in labeling:
    for j in range(len(eda_heart2[i].unique())):
        print(eda_heart2[i].unique()[j],round((len(eda_heart2[eda_heart2[i]==eda_heart2[i].unique()[j]]))/303,4))

In [ ]:
numeric_ft_dt['target']==numeric_ft_dt.target.astype('object')
dum=pd.get_dummies(numeric_ft_dt.target,prefix='target')
corr_numeric_dt=pd.concat([numeric_ft_dt,dum],axis=1).drop('target',axis=1)
corr_numeric_dt.head()

# numeric features correlation graph & scatter plot matrix checking

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(numeric_ft_dt.corr(),annot =True, cmap = 'Wistia')
plt.title('Heatmap for the Dataset',fontsize=20)
plt.show()

Positive correlation : max_heart_rate_achieved because blood vessels are narrowed and blood does not flow well, the heart beats a lot.

In [ ]:
sns.pairplot(numeric_ft_dt, hue='target')
plt.show()

# Test for independece(chi-squared test)

In [ ]:
association_dt=eda_heart1[['sex','chest_pain_type','fasting_blood_sugar','rest_ecg','exercise_induced_angina','st_slope','thalassemia','target']]
association_dt.astype('category')

In [ ]:
sex_crosstab=pd.crosstab(association_dt['sex'],association_dt.target,margins=True)
chest_pain_type_crosstab=pd.crosstab(association_dt['chest_pain_type'],association_dt.target,margins=True)
fasting_blood_sugar_crosstab=pd.crosstab(association_dt['fasting_blood_sugar'],association_dt.target,margins=True)
rest_ecg_crosstab=pd.crosstab(association_dt['rest_ecg'],association_dt.target,margins=True)
exercise_induced_angina_crosstab=pd.crosstab(association_dt['exercise_induced_angina'],association_dt.target,margins=True)
st_slope_crosstab=pd.crosstab(association_dt['st_slope'],association_dt.target,margins=True)
thalassemia_crosstab=pd.crosstab(association_dt['thalassemia'],association_dt.target,margins=True)

In [ ]:
from scipy.stats import chisquare, chi2_contingency, fisher_exact
stat, p, dof, expect=chi2_contingency(sex_crosstab.iloc[:-1,:-1])
print('test statistic : ',round(stat,4),'\n',
     'p_value : ',round(p,5),'\n',
     'freedom : ',dof,'\n',
     'Expected frequency : ',expect)
sex_crosstab

In [ ]:
stat, p, dof, expect=chi2_contingency(chest_pain_type_crosstab.iloc[:-1,:-1])
print('test statistic : ',round(stat,4),'\n',
     'p_value : ',round(p,5),'\n',
     'freedom : ',dof,'\n',
     'Expected frequency : ',expect)
chest_pain_type_crosstab

In [ ]:
stat, p, dof, expect=chi2_contingency(fasting_blood_sugar_crosstab.iloc[:-1,:-1])
print('test statistic : ',round(stat,4),'\n',
     'p_value : ',round(p,5),'\n',
     'freedom : ',dof,'\n',
     'Expected frequency : ',expect)
fasting_blood_sugar_crosstab

In [ ]:
stat, p, dof, expect=chi2_contingency(rest_ecg_crosstab.iloc[:-1,:-1])
print('test statistic : ',round(stat,4),'\n',
     'p_value : ',round(p,5),'\n',
     'freedom : ',dof,'\n',
     'Expected frequency : ',expect)

rest_ecg_crosstab

When That expected frequency < 5 is data proportion 1/4, Using Fisher exact test. but sometimes similar. haha

In [ ]:
stat, p, dof, expect=chi2_contingency(exercise_induced_angina_crosstab.iloc[:-1,:-1])
print('test statistic : ',round(stat,4),'\n',
     'p_value : ',round(p,5),'\n',
     'freedom : ',dof,'\n',
     'Expected frequency : ',expect)
exercise_induced_angina_crosstab

In [ ]:
stat, p, dof, expect=chi2_contingency(st_slope_crosstab.iloc[:-1,:-1])
print('test statistic : ',round(stat,4),'\n',
     'p_value : ',round(p,5),'\n',
     'freedom : ',dof,'\n',
     'Expected frequency : ',expect)
st_slope_crosstab

In [ ]:
stat, p, dof, expect=chi2_contingency(thalassemia_crosstab.iloc[:-1,:-1])
print('test statistic : ',round(stat,4),'\n',
     'p_value : ',round(p,5),'\n',
     'freedom : ',dof,'\n',
     'Expected frequency : ',expect)
thalassemia_crosstab

((sex, chest_pain_type, rest_ecg, exercise_induced_angina, st_slope, thalassemia), target) have association (p-value : 0.05)

# Model

## using randomforestclassifier

  ## How it work
 - Using bootstrap to extract samples(Trees)
 - Imlementation combines classifiers by averaging their probabilistic prediction, instead of
   letting each classifier vote for a single class.  
   
## Why use it
 - Individual decision trees typically exhibit high variance and tend to overfit. so randomforest
 achieve a reduced variance by combining diverse trees. but sometimes at the cost of a sli-
 ght increase in bias. The variance reduction is often significant hence yielding an overall
 better model.  
 
## When activating randomforest, what method using?
 - Gini index


In [ ]:
train_data=eda_heart1.copy()

In [ ]:
for i in ['sex','chest_pain_type','fasting_blood_sugar','rest_ecg','exercise_induced_angina','st_slope','thalassemia']:
    aa=pd.get_dummies(train_data[i])
    train_data=pd.concat([train_data,aa],axis=1)
    train_data=train_data.drop([i],axis=1)

In [ ]:
train_data=pd.concat([train_data.drop(['target'],axis=1),dt['target']],axis=1)
train_data['target']=train_data['target'].astype('category')

In [ ]:
X = train_data.drop(['target'],axis=1)
y = train_data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, stratify=y, random_state=123)

In [ ]:
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_train,y_train)

In [ ]:
export_graphviz(clf.estimators_[0],feature_names=X_train.columns,filled=True, out_file='tree.dot')

# 생성된 .dot 파일을 .png로 변환
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'decistion-tree.png', '-Gdpi=600'])

# jupyter notebook에서 .png 직접 출력
from IPython.display import Image
Image(filename = 'decistion-tree.png')

### Box Explain  
Classification Criteria ; G.I. , samples: number of samples, value: number of data, calss: more than target values names  
Although samples = sum(value), the value tends to be different because the bootstrap sample is currently being used. The reason is to use it to prevent learning of the same training set. In other words, when bootstrap is used, samples are extracted by the restoration extraction method and have randomness, and various DecisionTrees can be created. DecisionTrees (estimators=100) are classified and averaged to prevent overfitting of DecisionTrees, reducing variance.  

※ If you set the features to be used, you can calculate more various DecisionTree types.

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
cm=confusion_matrix(y_test,y_pred)

# Traing, Test data accuracy & Confusion Matrix(Heat map) & ROC curve

### confusion matrix
---

The classification result of the trained binary classification model can be summarized as the following 2x2 matrix, which is called a confusion matrix.
In this table, Actual Negative class and Actual Positive class represent the actual value of Target, and Predicted Negative class and Predicted Positive class represent the classification result of the classification model.
Here, negative indicates positive or true positive or false.
In general, in the error matrix, the true value of target is shown in the row and the classification result is shown in the column.

|  | **Predicted Negative class** | **Predicted Positive class** |
|:---|:---|:---|
|**Actual Negative class** | True Nagative(TN) | False Positive(FP) |
|**Actual Positive class** | False Negative(FN) | True Positive(TP) |

The meanings of TN (True Nagative), FP (False Positive), FN (False Negative), and TP (True Positive) described in each cell of the error matrix are as follows.

-True Positive (TP): The frequency of predicting the actual positive as a positive  
-False Positive (FP): The frequency of predicting that the actual negative is positive (the frequency of making a type 1 error)  
-False Negative (FN): The frequency of predicting that the actual positive is negative (the frequency of making a type 2 error)  
-True Negative (TN): The frequency of predicting the actual negative value as negative

Classification model performance evaluation scale  

## 1.Accuracy

  - A measure of the degree to which it is properly classified, and is the relative frequency of properly classified data.
  - Accuracy is an intuitive evaluation measure that indicates the performance of the model, but it is a measure that needs to be supplemented for unbalanced data. 
  
## 2.Precision

  - It is called precision or positive predictive value (PPV, positive predictive value).
  - It is the percentage of what is actually predicted as positive.
  - Precision is an appropriate measure of evaluation when it is desirable to reduce false positives. 

## 3. Recall (recall rate)
  - Also known as sensitivity, hit rate, and true positive rate (TPR).
  - It is the ratio of the data actually positive in the data classified as positive.
  - Recall rate is an appropriate evaluation measure when it is desirable to reduce False Negatives.
  - Reproducibility and precision have a negative correlation.

## 4. F1 score
  - It is calculated as the harmonic average of precision and recall.
  - Since precision and recall are considered together, unbalanced data is considered a more rational measure than accuracy.

## 5. Fall-out (false positive rate)
  - The proportion of positive among samples that do not belong to actual positive (the smaller the value, the better)

In [ ]:
print('training accuracy : ', round(clf.score(X_train,y_train),2))
print('test accuracy : ',clf.score(X_test,y_test),'\n')
print(classification_report(y_test,y_pred))
sns.heatmap(cm,annot =True, annot_kws = {'size':15},cmap='PuBu')
plt.title('confusion_matrix')
plt.xticks([0.5,1.5],('< 50% dn predict','> 50% dn predict'))
plt.yticks([0.5,1.5],('< 50% diameter narrowing','> 50% diameter narrowing'),rotation=0)

## Roc curve 
- The ROC curve is created by plotting the true positive rate(TPR) against the false positive rate(FPR) at various threshold setting.
- x axis : False Positive Rate(1 – specificity), y axis : True Positive Rate(sensitivity) 
- ROC curve can be generated by plotting the cumulative distribution function

![graph1](https://datascienceschool.net/upfiles/831d9dfbdcee499c9e32f39831d757f2.png)
![graph2](https://datascienceschool.net/upfiles/219c35d779eb4eb38be3fa0cc6b9ed38.png)

## [graph1]
To increase recall, you can lower the threshold to judge as positive so that even if there is only a little evidence, you can judge it as positive. But
In this case, even though it is negative, the sample data judged as positive increases as well, and the fallout increases at the same time. Conversely to lower the fallout
If the criterion for judging positive is strict, the number of sample data that has been judged negative due to lack of evidence increases as well, so
All.

## [graph2]
It is a graph drawn along TPR and FPR with the threshold of graph1 starting at the right end and going to the left end. => Roc Curve

## AUC (Area Under the Curve)
 - When the fallout is the same, the recall is large or the recall is the same, the smaller the fallout, the closer the AUC is to 1, which is a good model.

In [ ]:
y_pred_quant = clf.predict_proba(X_test)[:,1]

fpr,tpr,thresholds = roc_curve(y_test, y_pred_quant)
fig, ax = plt.subplots()
ax.plot(fpr,tpr)
ax.plot([0,1],[0,1],transform=ax.transAxes, ls="-",c='.3')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])

plt.rcParams['figure.figsize'] = (6,5)
plt.title('ROC curve for diabetes classifier',fontweight = 30)
plt.xlabel('False Positive Rate (1- Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid()
plt.show()
auc = auc(fpr,tpr)
print('AUC Score :',round(auc,2))

In [ ]:
# model accuracy
clf.score(X_test,y_test)

In [ ]:
# Feature importance
perm = PermutationImportance(clf,random_state=0).fit(X_test,y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist(),top=26)

I use data type float and dummies

In [ ]:
train_data2=dt.copy()
for i in ['sex','chest_pain_type','fasting_blood_sugar','rest_ecg','exercise_induced_angina','st_slope','thalassemia']:
    aa=pd.get_dummies(train_data2[i],prefix=i)
    train_data2=pd.concat([train_data2,aa],axis=1)
    train_data2=train_data2.drop([i],axis=1)
X = train_data2.drop(['target'],axis=1)
y = train_data2['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, stratify=y, random_state=123)
clf2 = RandomForestClassifier(max_depth=5, random_state=0)
clf2.fit(X_train,y_train)

In [ ]:
clf2.score(X_test,y_test)

 similiary score

The code below was created by referring to other notebooks. I will leave the source. If you need an explanation about it, you can check it there.

In [ ]:
base_features = train_data2.columns.values.tolist()
base_features.remove('target')
aa=['st_depression','num_major_vessels','age','resting_blood_pressure','cholesterol','max_heart_rate_achieved']
for i in range(len(aa)):
    feat_name = aa[i]
    pdp_dist = pdp.pdp_isolate(model=clf, dataset=X_test,
                               model_features = base_features,
                               feature = feat_name)

    pdp.pdp_plot(pdp_dist, feat_name)
plt.show()

In [ ]:
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values[1],X_test, plot_type='bar')

In [ ]:
shap.summary_plot(shap_values[1], X_test)

In [ ]:
def patient_analysis(model, patient):
  explainer = shap.TreeExplainer(model)
  shap_values = explainer.shap_values(patient)
  shap.initjs()
  return shap.force_plot(explainer.expected_value[1], shap_values[1], patient)

In [ ]:
patients = X_test.iloc[1,:].astype(float)
patient_analysis(clf, patients)

In [ ]:
patients = X_test.iloc[5,:].astype(float)
patient_analysis(clf, patients)

Check the difference with other models (net standard)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
neigh.fit(X_train,y_train)
neigh.score(X_test,y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,y_train)
gnb.score(X_test,y_test)

In [ ]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(X_train,y_train)
lsvc.score(X_test,y_test)

Thank you for reading my notebook

## reference

1. https://www.kaggle.com/tentotheminus9/what-causes-heart-disease-explaining-the-model

2. https://www.kaggle.com/roshansharma/heart-diseases-analysis
